In [1]:
import sys
sys.path.insert(0, "/home/04233/sw33286/AIDA-package")

In [4]:
from nyt_reader import NytReader
nyt_reader = NytReader(nyt_dir="/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_code/",
                       utils_dir="/work/04233/sw33286/AIDA-SCRIPTS",
                       dic_path="/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_info/indexer_word2emb_100k.p")

In [25]:
import random
import numpy as np

from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [19]:
# Preprocessing facilities

def sent_to_avg_emb(reader, sent):
    return np.array([reader.glove_embs[w_idx] for w_idx in sent]).mean(axis=0)

def doc_to_sent_avg_emb(reader, doc):
    return np.array([sent_to_avg_emb(reader, sent) for sent in doc])

def run_kmeans(reader, doc):
    km = KMeans(n_clusters=2).fit(doc_to_sent_avg_emb(reader, doc))
    return km.labels_

In [35]:
# Evaluation

def flip_clust(clust):
    return np.array([0 if i==1 else 1 for i in clust])

def clust_accuracy(true, pred):
    return max(accuracy_score(true, pred),
               accuracy_score(true, flip_clust(pred)))

def rand_evaluation(reader, k=1000):
    accuracies = []
    for _ in range(k):
        doc_mix, doc_labels = nyt_reader.get_rand_mixture()
        acc = clust_accuracy(doc_labels, run_kmeans(reader, doc_mix))
        accuracies.append(acc)
    return np.mean(accuracies)

In [36]:
%%time

n_round = 10
n_sample = 1000
avg_accs = []
for i in range(n_round):
    print('... Round {}'.format(i+1))
    avg_accs.append(rand_evaluation(nyt_reader, k=n_sample))
print('\n')
print('Average accuracy over {} samples = {}'.format(n_round*n_sample, np.mean(avg_accs)))

... Round 1
... Round 2
... Round 3
... Round 4
... Round 5
... Round 6
... Round 7
... Round 8
... Round 9
... Round 10


Average accuracy over 10000 samples = 0.746301973058876
CPU times: user 1min 38s, sys: 2.28 s, total: 1min 41s
Wall time: 3min 44s
